In [179]:
!pip install yfinance
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import json
import math
import boto3
import os
import shutil
import subprocess


# Sklearn

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Tensorflow / Keras

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [180]:
# -------------------------------
# 2) Add Technical Indicators
# -------------------------------
def add_indicators(df):
    delta = df["Close"].diff()
    gain = delta.clip(lower=0)
    loss = -1 * delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / (avg_loss + 1e-9)
    df["RSI"] = 100 - (100 / (1 + rs))
    ema12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD"] = ema12 - ema26
    df["MACD_SIGNAL"] = df["MACD"].ewm(span=9, adjust=False).mean()
    df["EMA7"] = df["Close"].ewm(span=7, adjust=False).mean()
    df["EMA14"] = df["Close"].ewm(span=14, adjust=False).mean()
    df["EMA21"] = df["Close"].ewm(span=21, adjust=False).mean()
    df["EMA50"] = df["Close"].ewm(span=50, adjust=False).mean()
    df["EMA100"] = df["Close"].ewm(span=100, adjust=False).mean()
    df["ATR"] = (df["High"] - df["Low"]).rolling(14).mean()
    df["BB_MID"] = df["Close"].rolling(20).mean()
    close_std = df["Close"].rolling(20).std().squeeze()
    df["BB_UPPER"] = df["BB_MID"] + 2 * close_std
    df["BB_LOWER"] = df["BB_MID"] - 2 * close_std
    df = df.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop=True)
    return df
# Example run
df = add_indicators(df)
print(df.head((25)))

Empty DataFrame
Columns: [(DATE, ), (Open, COCHINSHIP.NS), (High, COCHINSHIP.NS), (Low, COCHINSHIP.NS), (Close, COCHINSHIP.NS), (Volume, COCHINSHIP.NS), (RSI, ), (MACD, ), (MACD_SIGNAL, ), (EMA7, ), (EMA14, ), (EMA21, ), (EMA50, ), (EMA100, ), (ATR, ), (BB_MID, ), (BB_UPPER, ), (BB_LOWER, )]
Index: []


In [181]:
# -------------------------------
# 1) Fetch Stock Data
# -------------------------------
def get_stock_data(symbol, start_date, interval="1d"):
    original_symbol = symbol
    if not symbol.endswith(".NS"):
        symbol += ".NS"
    end_date = datetime.today().strftime("%Y-%m-%d")
    df = yf.download(symbol, start=start_date, end=end_date, interval=interval, auto_adjust=False)
    if df.empty:
        print(f"⚠ No data found for {symbol}")
        return None, original_symbol
    df = df[["Open", "High", "Low", "Close", "Volume"]].reset_index()
    df.rename(columns={"Date": "DATE"}, inplace=True)
    df["DATE"] = pd.to_datetime(df["DATE"]).dt.date
    return df, original_symbol
# Example run
df, stock_name = get_stock_data("INFY", "2021-01-01")
print(df.head((25)))

[*********************100%***********************]  1 of 1 completed

Price         DATE         Open         High          Low        Close  \
Ticker                  INFY.NS      INFY.NS      INFY.NS      INFY.NS   
0       2021-01-01  1257.900024  1265.500000  1255.800049  1260.449951   
1       2021-01-04  1269.000000  1290.000000  1261.150024  1288.250000   
2       2021-01-05  1282.000000  1299.000000  1275.050049  1293.800049   
3       2021-01-06  1300.000000  1302.000000  1268.050049  1282.099976   
4       2021-01-07  1296.000000  1297.650024  1255.500000  1262.150024   
5       2021-01-08  1278.250000  1316.800049  1278.250000  1312.099976   
6       2021-01-11  1342.099976  1383.349976  1340.000000  1376.199951   
7       2021-01-12  1378.000000  1378.000000  1360.000000  1371.750000   
8       2021-01-13  1373.849976  1392.800049  1360.000000  1387.150024   
9       2021-01-14  1360.000000  1384.000000  1318.449951  1370.500000   
10      2021-01-15  1360.000000  1364.449951  1340.000000  1344.949951   
11      2021-01-18  1338.000000  1340.

In [182]:

# -------------------------------
# 4) Prepare Sequences (Multi-output: Open + Close)
# -------------------------------
def prepare_sequences(df, lookback=30, feature_cols=None, target_cols=None):
    if feature_cols is None:
        feature_cols = ["Open", "High", "Low", "Volume", "Close",
                        "RSI", "MACD", "MACD_SIGNAL",
                        "EMA7", "EMA14", "EMA21", "EMA50", "EMA100",
                        "ATR", "BB_MID", "BB_UPPER", "BB_LOWER"]
    if target_cols is None:
        target_cols = ["Open", "Close"]
    # Scale features
    feature_scaler = MinMaxScaler()
    scaled_features = feature_scaler.fit_transform(df[feature_cols])
    # Scale targets (Open + Close)
    target_scaler = MinMaxScaler()
    scaled_target = target_scaler.fit_transform(df[target_cols])
    X, y = [], []
    for i in range(lookback, len(df)):
        X.append(scaled_features[i - lookback:i, :])
        y.append(scaled_target[i, :])   # Multi-output
    return np.array(X), np.array(y), feature_scaler, target_scaler, feature_cols, target_cols, df

In [183]:
# -------------------------------
# 5) Build LSTM Model (Multi-output)
# -------------------------------
def build_lstm(input_timesteps, n_features, n_outputs):
    model = Sequential([
        Input(shape=(input_timesteps, n_features)),
        LSTM(64, return_sequences=True),
        Dropout(0.3),
        LSTM(32),
        Dropout(0.3),
        Dense(n_outputs)   # Multi-output: Open + Close
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss="mean_squared_error")
    return model


In [184]:
# -------------------------------
# 6) Train, Evaluate, Predict (Multi-output)
# -------------------------------
# def train_eval_predict_lstm(df, lookback=60, test_size=0.2, future_days=3):
#     feature_cols = ["Open", "High", "Low", "Volume", "Close",
#                     "RSI", "MACD", "MACD_SIGNAL",
#                     "EMA7", "EMA14", "EMA21", "EMA50", "EMA100",
#                     "ATR", "BB_MID", "BB_UPPER", "BB_LOWER"]
#     target_cols = ["Open", "Close"]
#     X, y, feature_scaler, target_scaler, features, targets, df = prepare_sequences(
#         df, lookback, feature_cols, target_cols
#     )
#     split = int(len(X) * (1 - test_size))
#     X_train, X_test = X[:split], X[split:]
#     y_train, y_test = y[:split], y[split:]
#     model = build_lstm(lookback, X.shape[2], len(targets))
#     early_stopping = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)
#     lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, min_lr=1e-6, verbose=1)
#     model.fit(
#         X_train, y_train,
#         epochs=30,
#         batch_size=32,
#         validation_data=(X_test, y_test) if len(X_test) > 0 else None,
#         callbacks=[early_stopping, lr_scheduler],
#         verbose=1
#     )
#     if len(X_test) > 0:
#         y_pred_test = model.predict(X_test, verbose=0)
#         y_test_inv = target_scaler.inverse_transform(y_test)
#         y_pred_inv = target_scaler.inverse_transform(y_pred_test)
#         mae = mean_absolute_error(y_test_inv, y_pred_inv)
#         rmse = math.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
#         r2 = r2_score(y_test_inv, y_pred_inv)
#         print(f"📏 MAE: {mae:.4f}")
#         print(f"📉 RMSE: {rmse:.4f}")
#         print(f"📊 R² Score: {r2:.4f}")
#     else:
#         print("\n(No holdout test set due to short series; trained on all data.)")
#     # Retrain on all data
#     model.fit(X, y, epochs=early_stopping.patience, batch_size=32, verbose=0)
#     # Save MinMaxScaler values to local file
#     scaler_dict = {
#         "data_min": target_scaler.data_min_.tolist(),
#         "data_scale": target_scaler.scale_.tolist()
#     }
#     with open("scaler_params.json", "w") as f:
#         json.dump(scaler_dict, f)
    
#     # Upload to S3
#     bucket = "stockmarketprediction25"
#     s3 = boto3.client("s3")
#     s3.upload_file("scaler_params.json", bucket, "stock_lstm/scaler_params.json")
#     print("Scaler params saved to S3.")
    
#     # ✅ Save final trained model in SageMaker format
#     model_dir = "/home/sagemaker-user/model/1"
#     os.makedirs(model_dir, exist_ok=True)

#     model.export(model_dir)
#    # <-- CORRECT for SageMaker

#     print("✅ Model exported to:", export_path)
#     # ✅ Package into tar.gz for SageMaker
#     !tar -czvf model.tar.gz -C /home/sagemaker-user/model .

    
#     # Forecast future days
#     seq = X[-1].copy()
#     preds = []
#     for i in range(future_days):
#         next_scaled = model.predict(seq[np.newaxis, ...], verbose=0)[0]
#         next_values = target_scaler.inverse_transform([next_scaled])[0]  # (Open, Close)
#         preds.append(next_values)
#         next_row = seq[-1, :].copy()
#         next_row[features.index("Close")] = feature_scaler.transform(
#             [[0]*len(features)]
#         )[0, features.index("Close")]
#         seq = np.vstack([seq[1:], next_row])
    
#     # Upload Model to S3
#     prefix = "stock_lstm"
#     tar_path = "/home/sagemaker-user/model.tar.gz"
#     s3.upload_file(tar_path, bucket, f"{prefix}/model.tar.gz")
#     print(f"✅ Uploaded to s3://{bucket}/{prefix}/model.tar.gz")
    
#     return np.array(preds)



def train_eval_predict_lstm(df, lookback=60, test_size=0.2, future_days=3):

    feature_cols = ["Open", "High", "Low", "Volume", "Close",
                    "RSI", "MACD", "MACD_SIGNAL",
                    "EMA7", "EMA14", "EMA21", "EMA50", "EMA100",
                    "ATR", "BB_MID", "BB_UPPER", "BB_LOWER"]

    target_cols = ["Open", "Close"]

    X, y, feature_scaler, target_scaler, features, targets, df = prepare_sequences(
        df, lookback, feature_cols, target_cols
    )

    split = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    model = build_lstm(lookback, X.shape[2], len(targets))

    early_stopping = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, min_lr=1e-6)

    model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, lr_scheduler],
        verbose=1
    )

    if len(X_test) > 0:
        y_pred = model.predict(X_test, verbose=0)
        y_test_inv = target_scaler.inverse_transform(y_test)
        y_pred_inv = target_scaler.inverse_transform(y_pred)

        print("📏 MAE:", mean_absolute_error(y_test_inv, y_pred_inv))
        print("📉 RMSE:", math.sqrt(mean_squared_error(y_test_inv, y_pred_inv)))
        print("📊 R²:", r2_score(y_test_inv, y_pred_inv))

    # Retrain on full data
    model.fit(X, y, epochs=early_stopping.patience, batch_size=32, verbose=0)

    # Save scaler
    with open("scaler_params.json", "w") as f:
        json.dump({
            "data_min": target_scaler.data_min_.tolist(),
            "data_scale": target_scaler.scale_.tolist()
        }, f)

    bucket = "stockmarketprediction25"
    s3 = boto3.client("s3")
    s3.upload_file("scaler_params.json", bucket, "stock_lstm/scaler_params.json")

    # ======== CORRECT MODEL EXPORT (NO SERVO) ========
    model_root = "/home/sagemaker-user/model"
    model_dir = f"{model_root}/1"

    if os.path.exists(model_root):
        shutil.rmtree(model_root)

    os.makedirs(model_dir, exist_ok=True)

    model.export(model_dir)

    # Create tar.gz (Python-safe)
    subprocess.run(
        ["tar", "-czvf", "model.tar.gz", "-C", model_root, "."],
        check=True
    )

    s3.upload_file("model.tar.gz", bucket, "stock_lstm/model.tar.gz")

    # ======== FUTURE FORECAST (FIXED LOGIC) ========
    seq = X[-1].copy()
    preds = []

    for _ in range(future_days):
        next_scaled = model.predict(seq[np.newaxis, ...], verbose=0)[0]
        next_values = target_scaler.inverse_transform([next_scaled])[0]
        preds.append(next_values)

        next_row = seq[-1].copy()
        next_row[features.index("Open")] = next_scaled[targets.index("Open")]
        next_row[features.index("Close")] = next_scaled[targets.index("Close")]

        seq = np.vstack([seq[1:], next_row])

    return np.array(preds)



In [185]:
# -------------------------------
# 7) Main Execution
# -------------------------------
df, stock_name = get_stock_data("INFY", "2021-01-01")
if df is not None:
    df = add_indicators(df)
    csv_file_basic = f"{stock_name}_basic.csv"
    df.to_csv(csv_file_basic, index=False)
    print(f"✅ Data saved to {csv_file_basic}")
    next_3 = train_eval_predict_lstm(df, lookback=30, future_days=1)
    print("\n📈 Next Days Predicted (Open & Close):")
    for i, (op, cp) in enumerate(next_3, 1):
        print(f"Day {i}: 🟢 Open={op:.2f}, 🔴 Close={cp:.2f}")

[*********************100%***********************]  1 of 1 completed


✅ Data saved to INFY_basic.csv
Epoch 1/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 0.0742 - val_loss: 0.0098 - learning_rate: 0.0010
Epoch 2/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0207 - val_loss: 0.0067 - learning_rate: 0.0010
Epoch 3/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0152 - val_loss: 0.0040 - learning_rate: 0.0010
Epoch 4/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0143 - val_loss: 0.0038 - learning_rate: 0.0010
Epoch 5/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0126 - val_loss: 0.0036 - learning_rate: 0.0010
Epoch 6/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0128 - val_loss: 0.0032 - learning_rate: 0.0010
Epoch 7/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0120 - val_loss: 0.0031 - learning_rate: 0.0010
Epoch 8/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0110 - val_loss: 0.0030 - learning_rate: 0.0010
Epoch 9/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0106 - val_loss: 0.0031 - learning_r

INFO:tensorflow:Assets written to: /home/sagemaker-user/model/1/assets


Saved artifact at '/home/sagemaker-user/model/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30, 17), dtype=tf.float32, name='keras_tensor_78')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  139753518177872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139753518176720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139753518175568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139753518173648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139753518176912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139753518177488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139753518172880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139753518175952: TensorSpec(shape=(), dtype=tf.resource, name=None)
./
./1/
./1/variables/
./1/variables/variables.data-00000-of-00001
./1/variables/variables.index
./1/assets/
./1/saved_model.pb
./1/fingerprint.pb

In [186]:
# -------------------------------
# 8) Deploy Endpoint (SIMPLIFIED & WORKING)
# -------------------------------
import time
import sagemaker
from sagemaker.tensorflow import TensorFlowModel

bucket = "stockmarketprediction25"
prefix = "stock_lstm"

role = "arn:aws:iam::143063371745:role/service-role/AmazonSageMaker-ExecutionRole-20260105T124631"

model_uri = f"s3://{bucket}/{prefix}/model.tar.gz"

tf_model = TensorFlowModel(
    model_data=model_uri,
    role=role,
    framework_version="2.12",
    sagemaker_session=sagemaker.Session()
)

# ✅ Unique endpoint name (prevents ValidationException)
endpoint_name = f"stock-lstm-endpoint-{int(time.time())}"

predictor = tf_model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    endpoint_name=endpoint_name
)

print("✅ SageMaker Endpoint deployed:", endpoint_name)

# import sagemaker
# from sagemaker.tensorflow import TensorFlowModel

# bucket = "stockmarketprediction25"
# prefix = "stock_lstm"

# role = "arn:aws:iam::143063371745:role/service-role/AmazonSageMaker-ExecutionRole-20260105T124631"

# model_uri = f"s3://{bucket}/{prefix}/model.tar.gz"

# sess = sagemaker.Session()

# tf_model = TensorFlowModel(
#     model_data=model_uri,
#     role=role,
#     framework_version="2.12",
#     py_version="py310",
#     sagemaker_session=sess
# )

# # ✅ FIXED endpoint name (VERY IMPORTANT)
# endpoint_name = "stock-lstm-endpoint"

# predictor = tf_model.deploy(
#     initial_instance_count=1,
#     instance_type="ml.t2.medium",
#     endpoint_name=endpoint_name
# )

# print("✅ SageMaker Endpoint deployed:", endpoint_name)






-----!✅ SageMaker Endpoint deployed: stock-lstm-endpoint-1770215188


In [187]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

feature_cols = ["Open", "High", "Low", "Volume", "Close",
                "RSI", "MACD", "MACD_SIGNAL",
                "EMA7", "EMA14", "EMA21", "EMA50", "EMA100",
                "ATR", "BB_MID", "BB_UPPER", "BB_LOWER"]

lookback = 30

# Use your existing dataframe
data = df[feature_cols].values

# Temporary scaler (for endpoint input)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Last 30 timesteps
sample_input = scaled_data[-lookback:]
sample_input = sample_input.reshape(1, lookback, len(feature_cols))

print(sample_input.shape)



(1, 30, 17)


In [188]:

# Call endpoint
result = predictor.predict(sample_input.tolist())
print("Raw model output:", result)

# Extract predictions correctly
pred_scaled = np.array(result["predictions"])


Raw model output: {'predictions': [[0.577336252, 0.574987531]]}


In [189]:
# Load scaler params
import json
import boto3
import numpy as np

s3 = boto3.client("s3")
bucket = "stockmarketprediction25"
key = "stock_lstm/scaler_params.json"

obj = s3.get_object(Bucket=bucket, Key=key)
scaler_params = json.loads(obj["Body"].read())

data_min = np.array(scaler_params["data_min"])
scale = np.array(scaler_params["data_scale"])

pred_real = pred_scaled / scale + data_min
print("📈 Predicted prices (Open, Close):", pred_real)


📈 Predicted prices (Open, Close): [[1670.21019709 1669.76280262]]


In [190]:
#to generate “instances” automatically

import numpy as np
from sklearn.preprocessing import MinMaxScaler

feature_cols = ["Open", "High", "Low", "Volume", "Close",
                "RSI", "MACD", "MACD_SIGNAL",
                "EMA7", "EMA14", "EMA21", "EMA50", "EMA100",
                "ATR", "BB_MID", "BB_UPPER", "BB_LOWER"]

lookback = 30

data = df[feature_cols].values

scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)

instances = scaled[-lookback:].reshape(1, lookback, len(feature_cols)).tolist()

print(json.dumps({"instances": instances}, indent=2))


{
  "instances": [
    [
      [
        0.5692658204394088,
        0.5965612470871817,
        0.6041903261695016,
        0.43442133367737595,
        0.6005410217143026,
        0.8145893802189378,
        0.6724865125321969,
        0.6534824793697752,
        0.5325286387605304,
        0.5101063293918431,
        0.4881453928215065,
        0.43165140060962814,
        0.4462512766986473,
        0.3187452849064619,
        0.49800644743979117,
        0.5080083603937273,
        0.5230326484848402
      ],
      [
        0.5887589978273231,
        0.5782729451168569,
        0.5945980299578415,
        0.1239956476851739,
        0.5731032672799166,
        0.7070739598619542,
        0.6842046840846971,
        0.6631935366853854,
        0.5449720168682597,
        0.5212629175180017,
        0.49820018643218655,
        0.4385602429390767,
        0.45035605365668374,
        0.3011631929499933,
        0.5079886919592482,
        0.5184086015651419,
        0.531148127246

In [191]:
import json

payload = {
    "instances": instances  # your 1×30×17 tensor
}

print(json.dumps(payload))


{"instances": [[[0.5692658204394088, 0.5965612470871817, 0.6041903261695016, 0.43442133367737595, 0.6005410217143026, 0.8145893802189378, 0.6724865125321969, 0.6534824793697752, 0.5325286387605304, 0.5101063293918431, 0.4881453928215065, 0.43165140060962814, 0.4462512766986473, 0.3187452849064619, 0.49800644743979117, 0.5080083603937273, 0.5230326484848402], [0.5887589978273231, 0.5782729451168569, 0.5945980299578415, 0.1239956476851739, 0.5731032672799166, 0.7070739598619542, 0.6842046840846971, 0.6631935366853854, 0.5449720168682597, 0.5212629175180017, 0.49820018643218655, 0.4385602429390767, 0.45035605365668374, 0.3011631929499933, 0.5079886919592482, 0.5184086015651419, 0.5311481272466518], [0.5653671849618258, 0.563590705267583, 0.5896756208014735, 0.08032781721030009, 0.5667912424488346, 0.6563702422619362, 0.6888073922988077, 0.6719665355507647, 0.5526106788439531, 0.5299804359198643, 0.5066668245229551, 0.4448927791765871, 0.454216834526993, 0.2711035990140218, 0.5156366675340

In [192]:
import boto3

sm = boto3.client("sagemaker")
print(sm.list_endpoints()["Endpoints"])


[{'EndpointName': 'stock-lstm-endpoint-1770215188', 'EndpointArn': 'arn:aws:sagemaker:ap-south-1:143063371745:endpoint/stock-lstm-endpoint-1770215188', 'CreationTime': datetime.datetime(2026, 2, 4, 14, 26, 30, 252000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2026, 2, 4, 14, 29, 19, 160000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}, {'EndpointName': 'stock-lstm-endpoint-1770159338', 'EndpointArn': 'arn:aws:sagemaker:ap-south-1:143063371745:endpoint/stock-lstm-endpoint-1770159338', 'CreationTime': datetime.datetime(2026, 2, 3, 22, 55, 40, 587000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2026, 2, 3, 22, 57, 48, 345000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}, {'EndpointName': 'stock-lstm-endpoint-1770157904', 'EndpointArn': 'arn:aws:sagemaker:ap-south-1:143063371745:endpoint/stock-lstm-endpoint-1770157904', 'CreationTime': datetime.datetime(2026, 2, 3, 22, 31, 46, 461000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2026, 2, 3,

In [193]:
# import boto3

# sagemaker_client = boto3.client('sagemaker', region_name='ap-south-1')

# # List all endpoints
# response = sagemaker_client.list_endpoints(
#     SortBy='CreationTime',
#     SortOrder='Descending',
#     MaxResults=100
# )

# print("📋 Your Current Endpoints:")
# for idx, ep in enumerate(response['Endpoints'], 1):
#     status = ep['EndpointStatus']
#     name = ep['EndpointName']
#     created = ep['CreationTime']
#     print(f"{idx}. {name} - {status} - Created: {created}")

# # Delete old endpoints (CAREFUL - this deletes them!)
# print("\n🗑️ Deleting old endpoints...")
# for ep in response['Endpoints']:
#     endpoint_name = ep['EndpointName']
#     try:
#         sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
#         print(f"✅ Deleted: {endpoint_name}")
#     except Exception as e:
#         print(f"❌ Failed to delete {endpoint_name}: {str(e)}")

# print("\n⏳ Wait 30 seconds for deletion to complete...")
# import time
# time.sleep(30)

# # Now deploy your new endpoint
# print("\n🚀 Now deploying new endpoint...")

In [194]:
# import boto3

# sagemaker_client = boto3.client('sagemaker', region_name='ap-south-1')

# # List all endpoints
# response = sagemaker_client.list_endpoints(
#     StatusEquals='InService',  # Only show active endpoints
#     MaxResults=50
# )

# print("📋 Active SageMaker Endpoints:")
# for ep in response['Endpoints']:
#     print(f"✅ {ep['EndpointName']} - Status: {ep['EndpointStatus']}")

In [195]:
import boto3
import json
from datetime import datetime

# Create S3 client
s3 = boto3.client("s3", region_name="ap-south-1")

# Create metadata
metadata = {
    "stock_name": "INFY",
    "last_updated": datetime.utcnow().isoformat(),
    "model_type": "LSTM"
}

# Upload to S3
s3.put_object(
    Bucket="stockmarketprediction25",
    Key="stock_lstm/stock_metadata.json",
    Body=json.dumps(metadata, indent=2),
    ContentType="application/json"
)

print("✅ Metadata file created!")
print(f"📍 Location: s3://stockmarketprediction25/stock_lstm/stock_metadata.json")
print(f"📊 Stock Name: INFY")

✅ Metadata file created!
📍 Location: s3://stockmarketprediction25/stock_lstm/stock_metadata.json
📊 Stock Name: INFY


/tmp/ipykernel_241/3633056103.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "last_updated": datetime.utcnow().isoformat(),
